# Notebook 03: Fundamentos de k-means

En este notebook se va a implementar el algoritmo de clustering K-Means. En la primera parte se explica la implementación manual del algoritmo y luego se muestra como realizar los cálculos de forma sencilla con la librería sklearn.

In [ ]:
import numpy as np
import tensorflow as tf # Solamente lo utilizamos para descargar los datos
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
d1 = np.random.randn(20, 2) + 2
d2 = np.random.randn(20, 2) - 3
d3 = np.random.randn(20, 2)
d3[:, 0] = d3[:, 0] + 4
d3[:, 1] = d3[:, 1] - 4

In [ ]:
plt.plot(d1[:, 0], d1[:, 1], '.')
plt.plot(d2[:, 0], d2[:, 1], '.')
plt.plot(d3[:, 0], d3[:, 1], '.')
plt.show()

In [ ]:
datos = np.concatenate((d1, d2, d3), axis=0)
datos = datos[np.random.permutation(len(datos))]

## Implementación manual del algoritmo

In [ ]:
K = 3

In [ ]:
# Inicializar los centroides
# Se seleccionan puntos aleatorios del conjunto de datos


# While no se cumpla la condicion de parada do
# Condicion de parada: Que en una iteración del algoritmo no se modifique ningún centroide

    # Asignar cada dato xi al centroide más cercano

    # Actualizar los centroides según cierta operación
    # Operación: Actualizar el centroide por el promedio de los puntos del cluster


In [ ]:
clusters

In [ ]:
color = ["red", "green", "blue"]
for c in np.unique(clusters):
  plt.plot(datos[clusters == c, 0], datos[clusters == c, 1], '.', color=color[c], label="Cluster "+str(c+1), alpha=0.3)
  plt.scatter([centroides[c, 0]], [centroides[c, 1]], color=color[c], edgecolors="black")
plt.legend()
plt.show()

## K-Means usando la librería Sklearn

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
K = 3

kmeans = KMeans(n_clusters = K, init = 'random', n_init = 10).fit(datos)

In [ ]:
clusters = kmeans.predict(datos)
clusters

In [ ]:
color = ["red", "green", "blue"]
for c in np.unique(clusters):
  plt.plot(datos[clusters == c, 0], datos[clusters == c, 1], '.', color=color[c], label="Cluster "+str(c+1), alpha=0.3)
  plt.scatter([kmeans.cluster_centers_[c, 0]], [kmeans.cluster_centers_[c, 1]], color=color[c], edgecolors="black")
plt.legend()
plt.show()

### Frontera de clusterización con K-Means

In [ ]:
margen = 0.5
XX, YY = np.meshgrid(np.linspace(datos.min(axis=0)[0] - margen, datos.max(axis=0)[0] + margen, 100), np.linspace(datos.min(axis=0)[1] - margen, datos.max(axis=0)[1] + margen, 100))
points = np.concatenate([XX.reshape(-1, 1), YY.reshape(-1, 1)], axis=1)
labels = kmeans.predict(points)
labels = labels.reshape(XX.shape)

In [ ]:
color = ["red", "green", "blue"]
for c in np.unique(clusters):
  plt.plot(datos[clusters == c, 0], datos[clusters == c, 1], '.', color=color[c], label="Cluster "+str(c+1), alpha=0.5)
  plt.scatter([kmeans.cluster_centers_[c, 0]], [kmeans.cluster_centers_[c, 1]], color=color[c], edgecolors="black")
  plt.scatter(XX[labels == c], YY[labels == c], color=color[c], marker='.', alpha=0.08)
plt.show()

# K-Means para MNIST

Vamos a descargar el dataset de MNIST para hacer clustering con los datos de MNIST:

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Dividimos entre 255 para que todos los atributos de la imagen estén entre 0 y 1.

In [ ]:
x_train = x_train / 255
x_test = x_test / 255

Restamos ahora la media para destacar como positivos los píxeles de la clase y el resto con valores negativos.

In [ ]:
x_train = x_train - x_train.mean(axis=0)
x_test = x_test - x_train.mean(axis=0)

Veamos el aspecto de nuestros datos una vez más:

In [ ]:
plt.figure(figsize=(15,4))
for i in range(20):
  plt.subplot(2,10,i+1)
  plt.imshow(x_train[np.random.randint(60000)], cmap="bwr", vmin=-1, vmax=1)
plt.show()

In [ ]:
datos = x_train.reshape(x_train.shape[0], -1)
datos.shape

Para agilizar los cálculos (60000 imágenes tarda un buen rato) vamos a reducir el número de imágenes a 10000.

In [ ]:
datos = datos[:10000]
datos.shape

Empezamos con el algoritmo. Recuerda definir $K$.

- Probamos con K = 10, el número de clases

In [ ]:
K = 10

kmeans = KMeans(n_clusters = K, init = 'random', n_init = 10).fit(datos)
clusters = kmeans.predict(datos)
clusters

In [ ]:
plt.hist(clusters, bins=K)
plt.show()

In [ ]:
for c in np.unique(clusters):
  indices = np.where(clusters == c)[0]
  plt.figure(figsize=(15,4))
  plt.title("Cluster " + str(c))
  for i in range(10):
    plt.subplot(1,10,i+1)
    plt.imshow(x_train[indices][i], cmap="bwr", vmin=-1, vmax=1)
  plt.show()

In [ ]:
for center in kmeans.cluster_centers_:
  plt.figure(figsize=(2,2))
  plt.imshow(center.reshape(28,28), cmap="bwr", vmin=-1, vmax=1)
  plt.show()

**Discusión de los resultados**

No tiene sentido seleccionar un $K$ igual al número de clases. Estamos en clustering, no en aprendizaje supervisado!

**Ejercicio**: Busca un K razonable y cuéntanos qué conclusiones sacas.

In [ ]:
K = 30

kmeans = KMeans(n_clusters = K, init = 'random', n_init = 10).fit(datos)
clusters = kmeans.predict(datos)
clusters

In [ ]:
plt.hist(clusters, bins=K)
plt.show()

In [ ]:
for c in np.unique(clusters):
  indices = np.where(clusters == c)[0]
  if len(indices) > 10:
    plt.figure(figsize=(15,4))
    plt.title("Cluster " + str(c))
    for i in range(10):
      plt.subplot(1,10,i+1)
      plt.imshow(x_train[indices][i], cmap="bwr", vmin=-1, vmax=1)
    plt.show()

In [ ]:
for center in kmeans.cluster_centers_:
  plt.figure(figsize=(2,2))
  plt.imshow(center.reshape(28,28), cmap="bwr", vmin=-1, vmax=1)
  plt.show()

# K-Means para Breast Cancer

In [ ]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

In [ ]:
breastCancer = load_breast_cancer()
print(breastCancer.DESCR)

In [ ]:
datos = pd.DataFrame(breastCancer.data, columns=breastCancer.feature_names)
datos

## Preprocesado

In [ ]:
medias = datos.mean()
stds = datos.std()
datos = (datos - medias) / stds
datos

## Ejecución

In [ ]:
K = 10

kmeans = KMeans(n_clusters = K, init = 'random', n_init = 10).fit(datos)
clusters = kmeans.predict(datos)
clusters

In [ ]:
centroides = pd.DataFrame(kmeans.cluster_centers_, columns=datos.columns)
centroides

## Reducción de dimensionalidad para visualización

Recordad que PCA es una herramienta muy útil para reducir la dimensionalidad de vuestro problema. Además, se puede utilizar para intentar **proyectar un espacio N-dimensional a 2 dimensiones**. Vamos a utilizarlo.

In [ ]:
from sklearn.decomposition import PCA

Aunque nos va a dar igual, porque nuestro objetivo es proyectar a 2 dimensiones, conviene analizar la varianza explicada. Para ello calculamos un primer PCA con el número de componentes igual al número de atributos.

In [ ]:
pca = PCA(n_components=datos.shape[1])
pca.fit(datos)

Como se ve en la siguiente figura, PCA tiene una varianza explicada de aproximadamente un 65% con 2 componentes. **No es un buen resultado** pero queremos visualizar nuestros datos.

In [ ]:
plt.plot(pca.explained_variance_ratio_.cumsum())
plt.grid()
plt.show()

Ahora sí, calculamos PCA para 2 componentes y lo pintamos.

In [ ]:
pca = PCA(n_components=2)
pca.fit(datos)
datos_2d = pca.transform(datos)

Esta primera figura **No nos da mucha información**, no se ve a priori ningún grupo diferenciado en el dataset.

In [ ]:
plt.plot(datos_2d[:, 0], datos_2d[:, 1], '.', alpha=0.3)
plt.show()

Sin embargo, si incorporamos la información que nos proporciona K-Means, es decir, a qué cluster pertenece cada punto, podemos visualizar un poco mejor los datos.

- Hay grupos bien diferenciados (se ven claramente grupos de distinto color).

- Hay grupos que PCA no ha sido capaz de diferenciar, los agrupa todos en el mismo sitio pero K-Means es capaz de identificarlos.

- Hay grupos que en PCA están dispersados pero que K-Means los está reconociendo como cercanos.

In [ ]:
for c in np.unique(clusters):
  plt.plot(datos_2d[clusters==c, 0], datos_2d[clusters==c, 1], '.')
plt.show()

# Conclusión

- Hemos comprendido el funcionamiento de K-Means.

- Hemos implementado K-Means con selección aleatoria a mano.

- Hemos recordado el concepto de que al no estar en un problema supervisado, ajustar la K al número de clases que esperamos no tiene mucho sentido.

- Hemos comprobado la importancia de seleccionar una buena K.

- Hemos aprendido a usar PCA con clustering para visualizar en 2 dimensiones datos de alta dimensionalidad.